In [1]:
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from d2lvit import *
import copy
import torch.nn as nn

In [2]:
preprocess = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), preprocess) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), preprocess)
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 2000 Val size: 1000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [3]:
model = torch.load(os.path.join('..', 'models', 'd2lvit_3', 'digit-model.pt'))

In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.head = nn.Sequential(
    nn.LayerNorm((512,), eps=1e-05, elementwise_affine=True),
    nn.Linear(512, 45)
)
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.1494,  0.4578, -0.1679,  0.5008, -0.0109, -1.0668,  0.6952, -0.3902,
          0.2251, -0.6966,  0.0428, -0.7090,  0.7042, -0.2021,  0.4861, -0.4585,
          0.2272, -0.2714, -1.1671,  0.6746, -0.2831, -0.5771, -0.1215,  0.4273,
         -0.5943,  0.7270,  0.7594,  0.9642, -1.7082, -0.3811, -0.4800, -0.3453,
          0.9822,  0.3828, -1.0862, -0.2543, -0.5163,  0.2196, -0.0507, -0.4648,
         -0.1111,  0.8194, -0.3531,  0.3808, -0.4517]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'd2lvit_noprims'), digits_model, False, False)

Epoch 10 done, train loss: 0.0040 val acc: 1.0000
Epoch 20 done, train loss: 0.0013 val acc: 1.0000
Epoch 30 done, train loss: 0.0025 val acc: 1.0000
Epoch 40 done, train loss: 0.0006 val acc: 1.0000
Epoch 50 done, train loss: 0.0005 val acc: 1.0000
Epoch 60 done, train loss: 0.0002 val acc: 1.0000
Epoch 70 done, train loss: 0.0001 val acc: 1.0000
Epoch 80 done, train loss: 0.0023 val acc: 1.0000
Epoch 90 done, train loss: 0.0002 val acc: 1.0000
Epoch 100 done, train loss: 0.0001 val acc: 1.0000


In [ ]:
model = torch.load(os.path.join('..', 'models', 'd2lvit_noprims', 'tree-model.pt'))

In [10]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model, True, False)
print(train_acc)

1.0


In [11]:
val_acc = m.predict(model, val_loader, device, config, digits_model, True, False)
print(val_acc)

1.0
